In [1]:
import cv2
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import pandas as pd
import os
pd.set_option('display.max_colwidth', None)

### Crop test

### Dataframe for crop

In [2]:

def crop_save_labels(image_path, label_path):
    data = []
    
    # Load the image
    img = cv2.imread(str(image_path))
    # print('orignal size', img.shape)
    # Define the new size for resizing
    new_size = (640, 640)  # Example size, adjust as needed
    
    # Resize the image
    resized_img = cv2.resize(img, new_size, interpolation=cv2.INTER_LINEAR)
    
    # Read coordinates from the label file
    with open(label_path, "r") as label_file:
        for i, line in enumerate(label_file):
            try:
                # Parse the coordinates from the line
                class_, x_center, y_center, width, height = map(float, line.strip().split())
    
                # Convert normalized coordinates back to pixel values
                x_center_px = int(x_center * img.shape[1])
                y_center_px = int(y_center * img.shape[0])
                width_px = int(width * img.shape[1])
                height_px = int(height * img.shape[0])
    
                # Crop the region of interest
                cropped_img = img[y_center_px - height_px // 2:y_center_px + height_px // 2,
                                  x_center_px - width_px // 2:x_center_px + width_px // 2]
    
                # Resize the cropped image to 640 by 640
                resized_cropped_img = cv2.resize(cropped_img, new_size, interpolation=cv2.INTER_LINEAR)
                blur_result = is_blurred(resized_cropped_img)
                if not blur_result:

                    # Class definition
                    class_definition= {
                        '0': 'blue_cone',
                        '1': 'large_orange_cone',
                        '2': 'orange_cone',
                        '3': 'unknown_cone',
                        '4': 'yellow_cone'
                    }
                    # Save the images
                    class_name = class_definition.get(str(int(class_)))
                    image_file_extension = os.path.splitext(os.path.basename(image_path))[1]
                    image_file_name = os.path.splitext(os.path.basename(image_path))[0]
                    image_save_path = f"./data/images/{image_file_name}_{class_name}_{i}{image_file_extension}"
                    print("Writting:", image_save_path)
                    cv2.imwrite(image_save_path, resized_cropped_img)
    
                    # Save text file
                    label_save_path = f"./data/labels/{image_file_name}_{class_name}_{i}.txt"
                    
                    normalized_coords = normalize_image_coordinates()
        
                    # Add a row to the list
                    data.append([int(class_) ,class_name, normalized_coords[0], normalized_coords[1], normalized_coords[2], normalized_coords[3],
                                 image_save_path, label_save_path])
    
            except ValueError:
                print(f"Error parsing line: {line}")

    # Close all windows
    cv2.destroyAllWindows()
    # Create a DataFrame from the list
    df = pd.DataFrame(data, columns=['class_label' ,'class_name', 'x_center', 'y_center', 'width', 'height', 'image_path', 'label_path'])
    return df
    # # Display the DataFrame
    # print(df)


In [3]:
def create_destination_setup(img_directory_path, labels_directory_path):
    os.makedirs(img_directory_path, exist_ok=True)
    os.makedirs(labels_directory_path, exist_ok=True)
img_directory_path = "./data/images"
labels_directory_path = "./data/labels"
create_destination_setup(img_directory_path, labels_directory_path)

def normalize_image_coordinates():
    # Calculate normalized coordinates for the entire image
    x_center = 0.5
    y_center = 0.5
    w = 1.0
    h = 1.0
    return x_center, y_center, w, h

def is_blurred(image_array, threshold=1.90):
    if len(image_array.shape) == 3:
        img_gray = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)
    else:
        img_gray = image_array
    laplacian_var = cv2.Laplacian(img_gray, cv2.CV_64F).var()
    return laplacian_var < threshold

def get_image_label_pairs(image_dir, label_dir):
    image_label_pairs = []

    # List files in both image and label directories
    image_files = sorted(os.listdir(image_dir))
    label_files = sorted(os.listdir(label_dir))

    # Iterate through files to create pairs
    for image_file, label_file in zip(image_files, label_files):
        # Exclude files with ".rf." in their names
        if ".rf." in image_file or ".rf." in label_file:
            continue

        # Check if file names contain "_png" or "_jpg" for images and "_txt" for labels
        if ("_png" in image_file or "_jpg" in image_file) and ("_png" in label_file or "_jpg" in label_file):
            # Assuming the files have the same base name with different extensions
            image_path = os.path.join(image_dir, image_file)
            label_path = os.path.join(label_dir, label_file)

            # Check if both files exist before adding to pairs
            if os.path.isfile(image_path) and os.path.isfile(label_path):
                image_label_pairs.append((image_path, label_path))

    return image_label_pairs

def get_final_df(image_label_pairs):
    result_df = pd.DataFrame()
    for image_path, label_path in image_label_pairs:
        # print("Image Path:", image_path)
        # print("Label Path:", label_path)
        # print()
        df_sample = crop_save_labels(image_path, label_path)
        result_df = pd.concat([result_df, df_sample], ignore_index=True)
    return result_df

def write_annotations_to_text_file(dataframe):
    for index, row in dataframe.iterrows():
        class_label = row['class_label']
        x_center = row['x_center']
        y_center = row['y_center']
        width = row['width']
        height = row['height']
        label_path = row['label_path']

        # Construct the line
        line = f"{class_label} {x_center} {y_center} {width} {height}\n"

        # Write the line to the specified file
        with open(label_path, 'a') as file:
            file.write(line)

In [4]:
# Example usage:
image_directory = "../yolov5/data/train/images"
label_directory = "../yolov5/data/train/labels"

image_label_pairs = get_image_label_pairs(image_directory, label_directory)

In [5]:
# image_label_pairs

In [6]:
final_df = get_final_df(image_label_pairs)

Writting: ./data/images/BME_00020_jpg_blue_cone_5.jpg
Writting: ./data/images/BME_00020_jpg_yellow_cone_8.jpg
Writting: ./data/images/BME_00020_jpg_yellow_cone_9.jpg


/tmp/ipykernel_1345881/2063459427.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, df_sample], ignore_index=True)
/tmp/ipykernel_1345881/2063459427.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, df_sample], ignore_index=True)
/tmp/ipykernel_1345881/2063459427.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty o

Writting: ./data/images/BME_00041_jpg_yellow_cone_3.jpg


/tmp/ipykernel_1345881/2063459427.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, df_sample], ignore_index=True)
/tmp/ipykernel_1345881/2063459427.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, df_sample], ignore_index=True)


Writting: ./data/images/BME_00062_jpg_yellow_cone_38.jpg


...

Writting: ./data/images/ulm_00458_jpg_blue_cone_3.jpg
Writting: ./data/images/ulm_00458_jpg_large_orange_cone_4.jpg
Writting: ./data/images/ulm_00458_jpg_large_orange_cone_5.jpg
Writting: ./data/images/ulm_00462_jpg_blue_cone_0.jpg
Writting: ./data/images/ulm_00462_jpg_orange_cone_1.jpg
Writting: ./data/images/ulm_00462_jpg_orange_cone_3.jpg
Writting: ./data/images/ulm_00462_jpg_yellow_cone_4.jpg
Writting: ./data/images/ulm_00462_jpg_yellow_cone_5.jpg
Writting: ./data/images/ulm_00468_jpg_blue_cone_0.jpg
Writting: ./data/images/ulm_00468_jpg_blue_cone_1.jpg


/tmp/ipykernel_1345881/2063459427.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, df_sample], ignore_index=True)
/tmp/ipykernel_1345881/2063459427.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, df_sample], ignore_index=True)


Writting: ./data/images/ulm_00471_jpg_blue_cone_3.jpg
Writting: ./data/images/ulm_00471_jpg_orange_cone_5.jpg
Writting: ./data/images/ulm_00471_jpg_orange_cone_7.jpg
Writting: ./data/images/ulm_00471_jpg_yellow_cone_8.jpg
Writting: ./data/images/ulm_00471_jpg_yellow_cone_9.jpg
Writting: ./data/images/ulm_00475_jpg_blue_cone_3.jpg
Writting: ./data/images/ulm_00475_jpg_large_orange_cone_4.jpg
Writting: ./data/images/ulm_00475_jpg_orange_cone_5.jpg
Writting: ./data/images/ulm_00475_jpg_yellow_cone_8.jpg
Writting: ./data/images/ulm_00477_jpg_yellow_cone_0.jpg
Writting: ./data/images/ulm_00477_jpg_yellow_cone_1.jpg
Writting: ./data/images/ulm_00477_jpg_yellow_cone_2.jpg
Writting: ./data/images/ulm_00478_jpg_large_orange_cone_1.jpg
Writting: ./data/images/ulm_00479_jpg_large_orange_cone_0.jpg
Writting: ./data/images/ulm_00482_jpg_blue_cone_4.jpg
Writting: ./data/images/ulm_00482_jpg_large_orange_cone_5.jpg
Writting: ./data/images/ulm_00484_jpg_blue_cone_0.jpg
Writting: ./data/images/ulm_0048

/tmp/ipykernel_1345881/2063459427.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, df_sample], ignore_index=True)
/tmp/ipykernel_1345881/2063459427.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, df_sample], ignore_index=True)


Writting: ./data/images/ulm_00487_jpg_blue_cone_0.jpg
Writting: ./data/images/ulm_00487_jpg_blue_cone_1.jpg
Writting: ./data/images/ulm_00487_jpg_blue_cone_2.jpg
Writting: ./data/images/ulm_00487_jpg_orange_cone_4.jpg
Writting: ./data/images/ulm_00487_jpg_yellow_cone_10.jpg
Writting: ./data/images/ulm_00487_jpg_yellow_cone_11.jpg
Writting: ./data/images/ulm_00489_jpg_large_orange_cone_1.jpg
Writting: ./data/images/ulm_00489_jpg_orange_cone_2.jpg
Writting: ./data/images/ulm_00489_jpg_orange_cone_4.jpg
Writting: ./data/images/ulm_00491_jpg_blue_cone_0.jpg
Writting: ./data/images/ulm_00491_jpg_large_orange_cone_1.jpg
Writting: ./data/images/ulm_00492_jpg_blue_cone_0.jpg
Writting: ./data/images/ulm_00492_jpg_blue_cone_2.jpg
Writting: ./data/images/ulm_00492_jpg_blue_cone_3.jpg
Writting: ./data/images/ulm_00492_jpg_blue_cone_4.jpg
Writting: ./data/images/ulm_00492_jpg_large_orange_cone_5.jpg
Writting: ./data/images/ulm_00492_jpg_yellow_cone_6.jpg
Writting: ./data/images/ulm_00493_jpg_yellow

In [7]:
# final_df[final_df['image_path'].str.contains('.png')]
final_df

,class_label,class_name,x_center,y_center,width,height,image_path,label_path
0,0,blue_cone,0.5,0.5,1.0,1.0,./data/images/BME_00020_jpg_blue_cone_5.jpg,./data/labels/BME_00020_jpg_blue_cone_5.txt
1,4,yellow_cone,0.5,0.5,1.0,1.0,./data/images/BME_00020_jpg_yellow_cone_8.jpg,./data/labels/BME_00020_jpg_yellow_cone_8.txt
2,4,yellow_cone,0.5,0.5,1.0,1.0,./data/images/BME_00020_jpg_yellow_cone_9.jpg,./data/labels/BME_00020_jpg_yellow_cone_9.txt
3,4,yellow_cone,0.5,0.5,1.0,1.0,./data/images/BME_00041_jpg_yellow_cone_3.jpg,./data/labels/BME_00041_jpg_yellow_cone_3.txt
4,4,yellow_cone,0.5,0.5,1.0,1.0,./data/images/BME_00062_jpg_yellow_cone_38.jpg,./data/labels/BME_00062_jpg_yellow_cone_38.txt
...,...,...,...,...,...,...,...,...
11127,1,large_orange_cone,0.5,0.5,1.0,1.0,./data/images/ulm_00498_jpg_large_orange_cone_5.jpg,./data/labels/ulm_00498_jpg_large_orange_cone_5.txt
11128,0,blue_cone,0.5,0.5,1.0,1.0,./data/images/ulm_00499_jpg_blue_cone_0.jpg,./data/labels/ulm_00499_jpg_blue_cone_0.txt
11129,4,yellow_cone,0.5,0.5,1.0,1.0,./data/images/ulm_00499_jpg_yellow_cone_1.jpg,./data/labels/ulm_00499_jpg_yellow_cone_1.txt
11130,4,yellow_cone,0.5,0.5,1.0,1.0,./data/images/ulm_00499_jpg_yellow_cone_2.jpg,./data/labels/ulm_00499_jpg_yellow_cone_2.txt


In [8]:
write_annotations_to_text_file(final_df)

In [9]:
### Savev df to csv
final_df.to_csv("cropped_640_by_640.csv", )

### Testing out dataframes

In [10]:
final_df.groupby('class_label').size().reset_index(name='Count')

,class_label,Count
0,0,4841
1,1,1197
2,2,1358
3,3,33
4,4,3703


### Test the coordinates

In [ ]:
# Iterate through the rows of the DataFrame
for i, row in final_df.iterrows():
    # Read the image
    print(row['image_path'])
    img = cv2.imread(row['image_path'])

    # Get normalized coordinates
    x_center_norm = row['x_center']
    y_center_norm = row['y_center']
    width_norm = row['width']
    height_norm = row['height']

    # Calculate bounding box coordinates in pixel values
    x_center_px = int(x_center_norm * img.shape[1])
    y_center_px = int(y_center_norm * img.shape[0])
    width_px = int(width_norm * img.shape[1])
    height_px = int(height_norm * img.shape[0])

    # Calculate the top-left corner of the bounding box
    x_top_left_px = int(x_center_px - width_px / 2)
    y_top_left_px = int(y_center_px - height_px / 2)

    # Create a rectangle patch
    rect = patches.Rectangle(
        (x_top_left_px, y_top_left_px),
        width_px, height_px,
        linewidth=5, edgecolor='r', facecolor='none'
    )
    # Create a figure and axes outside the loop
    fig, ax = plt.subplots()
    ax.add_patch(rect)

    # Display the image with bounding box using plt
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    # Show the plot
    plt.show()